In [1]:
# Particle Swarm Optimization (PSO) Code%% Girdiler
minimization = 1             # 0: maximization, 1: minimization
coeff        = 1.0           # katsayı // 1.0
w            = 0.0196        # weight of the partical velocity  //w    = 0.0196;
eta1         = 3.9804        # eta1 = 3.9804 // partical best
eta2         = 0.2157        # eta2 = 0.2157   // global best
eta3         = 3.2353        # eta3 = 3.2353  // closest neighbour
eta4         = 0.5098        # eta4 = 0.5098 // closest best neighbour
eta5         = 0.00          # eta5 = 0.00 // average position of the swarm
eta6         = 0.00          # eta6 = 0.00 // average velocity of the swarm
#fitness_function = @egg;    # the function that will be optimized
number_of_particles = 150    # The number of the elements (particles/bees/ants/birds) inside the swarm
number_of_variables = 2      # variable sayısı / dimension sayısı
migration           = 1      # 0: no-migration, 1: migration
pmig                = 0.15   # migration probability if migration is allowed // 0.15
mig_katsayi         = 0.0110 # migration katsayısı // = pmig (0.15)
objective_number    = 1      # the number of the objectives that the fitness function gives out
iteration           = 200    # Number of iterations
saved_data          = 1      # 1: use saved x_encoded values, 0: start randomly

In [ ]:
lower_boundaries = zeros(1,number_of_variables);
upper_boundaries = zeros(1,number_of_variables);

lower_boundaries(1,:) = [-512 -512];
upper_boundaries(1,:) = [512 512];
% lower_boundaries(1,:) = 0 * ones(1,number_of_variables);
% upper_boundaries(1,:) = 100 * ones(1,number_of_variables);
vel_min = -20;
vel_max = 20;

position = zeros(number_of_particles,number_of_variables);
velocity = zeros(number_of_particles,number_of_variables);
[row col] = size(position);          %%% size of the domain row: number of particles, col: number of variables
globalbest_value(1,1:col) = 1000000;

distances = zeros(number_of_particles,number_of_particles);
distances(1:number_of_particles,1:number_of_particles*number_of_variables) = 0;
closest_distances(1:number_of_particles,1:4*number_of_variables) = 1000000000;     %%% Her bir particle'�n kendisi dahil en yak�n 4 tane kom�usu i�in mesafeler bulunuyor.
closest_particles(1:number_of_particles,1:4*number_of_variables) = 0;

%%%% Kaydedilmi� veri kullan�lmadan, rastgele bir ver iseti ile ba�lan�yor %%%%

if(saved_data == 0)
    %%
    %%% Her bir partical i�in rastgele pozisyon (x1, x2, ..., xn)  ve h�z
    %%% atamas� yap�l�yor.
    for i = 1:row
        for j = 1:col
            position(i,j) = (upper_boundaries(1,j) - lower_boundaries(1,j)) * random('unif',0,1) + lower_boundaries(1,j);
            velocity(i,j) = random('unif',0,1);
        end
        fitness_values(i,:) = fitness_function(position(i,:));
    end
    personal_best_pos = position;
    
    total_velocity   = zeros(1,col);
    total_position   = zeros(1,col);
    average_velocity = zeros(1,col);
    average_position = zeros(1,col);
    for j = 1:col
        for i = 1:row
            total_velocity(1,j) = total_velocity(1,j) + velocity(i,j);
            total_position(1,j) = total_position(1,j) + position(i,j);
        end
        average_velocity(1,j) = total_velocity(1,j) / row;
        average_position(1,j) = total_position(1,j) / row;
    end
    
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%
    %%% Her bir partical'�n di�er partical'la ile aras�ndaki mesafe her bir variable i�in ayr� ayr� bulunuyor %%%
    for j = 1:col
        for i = 1:row
            for k = 1:row
                column = (j-1)*row + k;
                distances(i,column) = abs(position(i,j) - position(k,j));
            end
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%
    %%% Her bir particle'�n kendisi dahil en yak�n 4 tane kom�usu i�in
    %%% mesafeler ve partical indisleri bulunuyor. %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    for j = 1:col
        for i=1:row
            column = (j-1)*row ;
            sorted_neighbours(i,column+1:column+row) = sort(distances(i,column+1:column+row));
        end
    end
    
    for j = 1:col
        for i=1:row
            column = (j-1)*row ;
            sutun = (j-1) * 4;
            closest_distances(i,sutun+1:sutun+4) = sorted_neighbours(i,column+1:column+4);
        end
    end
    for j = 1:col
        for i=1:row
            for k=1:4
                column = (j-1)*row;
                sutun = (j-1) * 4 + k;
                closest_particles(i,sutun) = find(closest_distances(i,sutun) == distances(i,column+1:column+row),1,'first');
            end
        end
    end
    
    %%
    %%% Her bir partical i�in, o zaman kadar elde edilmi� en iyi fitness
    %%% de�eri kaydediliyor.
    for ii=1:row
        partical_best(ii,:) = fitness_function(position(ii,:));
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%%%%% En yak�n 4 particle'�n fitness de�erleri yaz�l�yor %%%%%%%%%%%%%
    for j = 1:col
        for i=1:row
            for k=1:4
                sutun = (j-1) * 4 + k;
                closest_particles_fitness_values(i,sutun) = fitness_values(closest_particles(i,sutun),1);
            end
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%%%% En yak�n 4 partical'�n fitness de�erleri ve particle'lar�n indisleri her bir variable i�in
    %%%%% en alttan en yukar�ya do�ru s�ralan�yor
    
    %%% En yak�n particle'lar�n fitness de�erleri s�ralan�yor
    closest_particles_fitness_values_sorted(:,:) = closest_particles_fitness_values(:,:);
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            sorted_fitness_values(i,sutun+2:sutun+4) = sort(closest_particles_fitness_values(i,sutun+2:sutun+4));
        end
    end
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            closest_particles_fitness_values_sorted(i,sutun+2:sutun+4) = sorted_fitness_values(i,sutun+2:sutun+4);
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%% En yak�n particle'lar�n indisleri s�ralan�yor
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            closest_best_particles_indices(i,j) = closest_particles(i,find(closest_particles_fitness_values_sorted(i,sutun+2)==closest_particles_fitness_values(i,sutun+1:sutun+4),1,'first')+sutun);
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%
    %%%%% Global en iyi fitness de�eri bulunuyor
    if(minimization == 1)
        for ii=1:row
            if (partical_best(ii,:) < globalbest_value)
                globalbest_value = partical_best(ii,:);
                global_best_index = ii;
                global_best_position = position(global_best_index,:);
            end
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    genn = 1;
    %%
    %%% kaydedilmi� veri kullan�l�yor
else
    load('PSO_saved_data.mat');
    [saved_data_birey_sayisi saved_data_col] = size(fitness_values);
    genn = genn_saved;
    if(row>saved_data_birey_sayisi)
        for i = saved_data_birey_sayisi+1:row
            for j = 1:col
                position(i,j) = (upper_boundaries(1,j) - lower_boundaries(1,j)) * random('unif',0,1) + lower_boundaries(1,j);
                velocity(i,j) = random('unif',0,1);
            end
            fitness_values(i,:) = fitness_function(position(i,:));
            personal_best_pos(i,:) = position(i,:);
        end
        %%
        %%% Her bir partical'�n di�er partical'la ile aras�ndaki mesafe her bir variable i�in ayr� ayr� bulunuyor %%%
        for j = 1:col
            for i = 1:row
                for k = 1:row
                    column = (j-1)*row + k;
                    distances(i,column) = abs(position(i,j) - position(k,j));
                end
            end
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        %%
        %%% Her bir particle'�n kendisi dahil en yak�n 4 tane kom�usu i�in
        %%% mesafeler ve partical indisleri bulunuyor. %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        for j = 1:col
            for i=1:row
                column = (j-1)*row ;
                sorted_neighbours(i,column+1:column+row) = sort(distances(i,column+1:column+row));
            end
        end
        
        for j = 1:col
            for i=1:row
                column = (j-1)*row ;
                sutun = (j-1) * 4;
                closest_distances(i,sutun+1:sutun+4) = sorted_neighbours(i,column+1:column+4);
            end
        end
        for j = 1:col
            for i=1:row
                for k=1:4
                    column = (j-1)*row;
                    sutun = (j-1) * 4 + k;
                    closest_particles(i,sutun) = find(closest_distances(i,sutun) == distances(i,column+1:column+row),1,'first');
                end
            end
        end
        
        %%
        %%% Her bir partical i�in, o zaman kadar elde edilmi� en iyi fitness
        %%% de�eri kaydediliyor.
        for ii=saved_data_birey_sayisi+1:row
            partical_best(ii,:) = fitness_function(position(ii,:));
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        %%%%%% En yak�n 4 particle'�n fitness de�erleri yaz�l�yor %%%%%%%%%%%%%
        for j = 1:col
            for i=1:row
                for k=1:4
                    sutun = (j-1) * 4 + k;
                    closest_particles_fitness_values(i,sutun) = fitness_values(closest_particles(i,sutun),1);
                end
            end
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        %%%%% En yak�n 4 partical'�n fitness de�erleri ve particle'lar�n indisleri her bir variable i�in
        %%%%% en alttan en yukar�ya do�ru s�ralan�yor
        
        %%% En yak�n particle'lar�n fitness de�erleri s�ralan�yor
        closest_particles_fitness_values_sorted(:,:) = closest_particles_fitness_values(:,:);
        for j = 1:col
            for i=1:row
                sutun = (j-1) * 4 ;
                sorted_fitness_values(i,sutun+2:sutun+4) = sort(closest_particles_fitness_values(i,sutun+2:sutun+4));
            end
        end
        for j = 1:col
            for i=1:row
                sutun = (j-1) * 4 ;
                closest_particles_fitness_values_sorted(i,sutun+2:sutun+4) = sorted_fitness_values(i,sutun+2:sutun+4);
            end
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        %%% En yak�n particle'lar�n indisleri s�ralan�yor
        for j = 1:col
            for i=1:row
                sutun = (j-1) * 4 ;
                closest_best_particles_indices(i,j) = closest_particles(i,find(closest_particles_fitness_values_sorted(i,sutun+2)==closest_particles_fitness_values(i,sutun+1:sutun+4),1,'first')+sutun);
            end
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        %%
        %%%%% Global en iyi fitness de�eri bulunuyor
        if(minimization == 1)
            for ii=1:row
                if (partical_best(ii,:) < globalbest_value)
                    globalbest_value = partical_best(ii,:);
                    global_best_index = ii;
                    global_best_position = position(global_best_index,:);
                end
            end
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    end
end

scatterplot(position(:,1:2));

iter = 0;

%%
%%%%%%% iterasyonlar ba�l�yor
while(iter < iteration)
    remainder = rem(iter,10);
    %%
    %%% Pozisyon ve h�z de�erleri g�ncelleniyor.
    for ii=1:row
        
        for jj=1:col
            velocity(ii,jj) = coeff * (w * velocity(ii,jj) ...
                + eta1 * random('unif',0,1) * (personal_best_pos(ii,jj) - position(ii,jj))...
                + eta2 * random('unif',0,1) * (global_best_position(:,jj) - position(ii,jj)))...
                + eta3 * random('unif',0,1) * (position(closest_particles(ii,(jj-1)*4+2),jj) - position(ii,jj))...
                + eta4 * random('unif',0,1) * (position(closest_best_particles_indices(ii,jj) ,jj) - position(ii,jj))...
                + eta5 * random('unif',0,1) * (average_position(1,jj) - position(ii,jj))...
                + eta6 * random('unif',0,1) * average_velocity(1,jj);
            
            if(velocity(ii,jj)>vel_max)
                velocity(ii,jj) = vel_max;
            elseif(velocity(ii,jj)<vel_min)
                velocity(ii,jj) = vel_min;
            end
            
            position(ii,jj) = position(ii,jj) + velocity(ii,jj);
            
            if(position(ii,jj)>upper_boundaries(1,jj))
                position(ii,jj) = upper_boundaries(1,jj);
            elseif(position(ii,jj)<lower_boundaries(1,jj))
                position(ii,jj) = lower_boundaries(1,jj);
            end
        end
        
        fitness_values(ii,:) = fitness_function(position(ii,:));
        
        if(fitness_values(ii,:) < partical_best(ii,:))
            partical_best(ii,:) = fitness_values(ii,:);
            personal_best_pos(ii,:) = position(ii,:);
        end
        
        if (partical_best(ii,:) < globalbest_value)
            globalbest_value = partical_best(ii,:);
            global_best_index = ii;
            global_best_position = position(global_best_index,:);
        end
        
        total_velocity   = zeros(1,col);
        total_position   = zeros(1,col);
        average_velocity = zeros(1,col);
        average_position = zeros(1,col);
        for j = 1:col
            for i = 1:row
                total_velocity(1,j) = total_velocity(1,j) + velocity(i,j);
                total_position(1,j) = total_position(1,j) + position(i,j);
            end
            average_velocity(1,j) = total_velocity(1,j) / row;
            average_position(1,j) = total_position(1,j) / row;
        end
    end
    %%
    %%%% Migration eklemesi yap�l�yor e�er migration=1 ise %%%
    if(migration==1)
        migration_probability = random('unif',0,1);
        total_migrating_particle_number = round(pmig*number_of_particles+objective_number);
        migrating_particle_number1 = round((pmig*number_of_particles+objective_number)/4);
        migrating_particle_number2 = 2 * migrating_particle_number1;
        migrating_particle_number3 = 3 * migrating_particle_number1;
        if(migration_probability <= pmig)
            sorted = sort(fitness_values(:,objective_number),'descend');
            
            %%% Fitness de�eri en d���k olan particle'lar�n indeksleri
            %%% bulunuyor
            for wi=1:total_migrating_particle_number
                worst_indeks(wi,1) = find(fitness_values(:,objective_number)==sorted(wi,1),1,'first');
            end
            
            %%% Fitness de�eri en y�ksek olan particle'lar�n indeksleri
            %%% bulunuyor
            for bi=1:(0.20*number_of_particles)
                best_indeks(bi,1) = find(fitness_values(:,objective_number)==sorted(number_of_particles+1-bi,1),1,'first');
                best_particle_array(bi,:) = position(best_indeks(bi,1),:);
            end
            
            %%% Fitness de�eri en y�ksek particle'lar�n average
            %%% pozisyonlar� bulunuyor.
            for j=1:col
                best_particles_average_position(1,j) = averaging(best_particle_array(:,j));
            end
            
            %%% En k�t� fitness de�erine sahip olanlar�n yar�s�, random
            %%% particle'lar ile de�i�tiriliyor
            for i = 1:migrating_particle_number1
                for j = 1:col
                    position(worst_indeks(i),j) = (upper_boundaries(1,j) - lower_boundaries(1,j)) * random('unif',0,1) + lower_boundaries(1,j);
                    velocity(worst_indeks(i),j) = random('unif',0,1);
                end
            end
            
            %%% En k�t� fitness de�erine sahip olanlar�n kalan yar�s�, en
            %%% iyi particle'lar�n ortalama pozisyon ve h�z de�eri merkez
            %%% al�narak random pozisyon ve h�z de�erlerine sahip yeni
            %%% particle'lar ile de�i�tiriliyor.
            for i = migrating_particle_number1+1:migrating_particle_number2
                for j = 1:col
                    sign = 1;
                    rastgelelik = random('unif',0,1);
                    if(rastgelelik < 0.50)
                        sign = -1;
                    end
                    position(worst_indeks(i),j) = 0.10 * (upper_boundaries(1,j) - lower_boundaries(1,j)) * random('unif',0,1) + sign * best_particles_average_position(1,j);
                    velocity(worst_indeks(i),j) = random('unif',0,1) * average_velocity(1,j);
                    
                    if(position(worst_indeks(i),j)>upper_boundaries(1,j))
                        position(worst_indeks(i),j) = upper_boundaries(1,j);
                    elseif(position(worst_indeks(i),j)<lower_boundaries(1,j))
                        position(worst_indeks(i),j) = lower_boundaries(1,j);
                    end
                    
                end
            end
            for i = migrating_particle_number2+1:migrating_particle_number3
                for j = 1:col
                    sign = 1;
                    rastgelelik = random('unif',0,1);
                    if(rastgelelik < 0.50)
                        sign = -1;
                    end
                    position(worst_indeks(i),j) = 10 * (best_particle_array(1,j) - best_particle_array(end,j)) * random('unif',0,1) + sign * best_particles_average_position(1,j);
                    velocity(worst_indeks(i),j) = random('unif',0,1) * average_velocity(1,j);
                    
                    if(position(worst_indeks(i),j)>upper_boundaries(1,j))
                        position(worst_indeks(i),j) = upper_boundaries(1,j);
                    elseif(position(worst_indeks(i),j)<lower_boundaries(1,j))
                        position(worst_indeks(i),j) = lower_boundaries(1,j);
                    end
                    
                end
            end
            for i = migrating_particle_number3+1:total_migrating_particle_number
                for j = 1:col
                    sign = 1;
                    rastgelelik = random('unif',0,1);
                    if(rastgelelik < 0.50)
                        sign = -1;
                    end
                    position(worst_indeks(i),j) = 2.0 * (best_particles_average_position(1,j) - average_position(1,j)) * random('unif',0,1) + sign * global_best_position(1,j);
                    velocity(worst_indeks(i),j) = random('unif',0,1) * average_velocity(1,j);
                    
                    if(position(worst_indeks(i),j)>upper_boundaries(1,j))
                        position(worst_indeks(i),j) = upper_boundaries(1,j);
                    elseif(position(worst_indeks(i),j)<lower_boundaries(1,j))
                        position(worst_indeks(i),j) = lower_boundaries(1,j);
                    end
                    
                end
            end
            
        end
        clear worst_indeks;
    end
    %%
    %%% Her bir partical'�n di�er partical'la ile aras�ndaki mesafe her bir variable i�in ayr� ayr� bulunuyor %%%
    for j = 1:col
        for i = 1:row
            for k = 1:row
                column = (j-1)*row + k;
                distances(i,column) = abs(position(i,j) - position(k,j));
            end
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%
    %%% Her bir particle'�n kendisi dahil en yak�n 4 tane kom�usu i�in
    %%% mesafeler ve partical indisleri bulunuyor. %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    for j = 1:col
        for i=1:row
            column = (j-1)*row ;
            sorted_neighbours(i,column+1:column+row) = sort(distances(i,column+1:column+row));
        end
    end
    
    for j = 1:col
        for i=1:row
            column = (j-1)*row ;
            sutun = (j-1) * 4;
            closest_distances(i,sutun+1:sutun+4) = sorted_neighbours(i,column+1:column+4);
        end
    end
    for j = 1:col
        for i=1:row
            for k=1:4
                column = (j-1)*row;
                sutun = (j-1) * 4 + k;
                closest_particles(i,sutun) = find(closest_distances(i,sutun) == distances(i,column+1:column+row),1,'first');
            end
        end
    end
    
    %%
    %%% Her bir partical i�in, o zaman kadar elde edilmi� en iyi fitness
    %%% de�eri kaydediliyor.
    for ii=1:row
        partical_best(ii,:) = fitness_function(position(ii,:));
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%%%%% En yak�n 4 particle'�n fitness de�erleri yaz�l�yor %%%%%%%%%%%%%
    for j = 1:col
        for i=1:row
            for k=1:4
                sutun = (j-1) * 4 + k;
                closest_particles_fitness_values(i,sutun) = fitness_values(closest_particles(i,sutun),1);
            end
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%%%% En yak�n 4 partical'�n fitness de�erleri ve particle'lar�n indisleri her bir variable i�in
    %%%%% en alttan en yukar�ya do�ru s�ralan�yor
    
    %%% En yak�n particle'lar�n fitness de�erleri s�ralan�yor
    closest_particles_fitness_values_sorted(:,:) = closest_particles_fitness_values(:,:);
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            sorted_fitness_values(i,sutun+2:sutun+4) = sort(closest_particles_fitness_values(i,sutun+2:sutun+4));
        end
    end
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            closest_particles_fitness_values_sorted(i,sutun+2:sutun+4) = sorted_fitness_values(i,sutun+2:sutun+4);
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%% En yak�n particle'lar�n indisleri s�ralan�yor
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            closest_best_particles_indices(i,j) = closest_particles(i,find(closest_particles_fitness_values_sorted(i,sutun+2)==closest_particles_fitness_values(i,sutun+1:sutun+4),1,'first')+sutun);
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    %%
    
    if (remainder == 1)
        scatterplot(position(:,1:2));
    end
    
    best_value_array(genn,1) = globalbest_value;
    
    genn = genn + 1;
    iter = iter + 1;
    text1 = 'iter = %d\n';
    fprintf(text1,iter);
    
end
genn_saved = genn;
save('PSO_saved_data.mat','position','velocity','closest_distances','closest_particles','globalbest_value','global_best_position','partical_best','personal_best_pos','closest_best_particles_indices','average_velocity','average_position','fitness_values','genn_saved','best_value_array');
close all;
scatterplot(position(:,1:2));